In [2]:
!pip install arxiv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=50a8fd172cab5123e036d28e72521456000d8856c44687797e6596f93063965d
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   -------------------------- ------------- 2/3 [arxiv]
   ---------------------------------------- 3/3 [arxiv]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [17]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200 )
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [18]:
wiki.name

'wikipedia'

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb= FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000248CB70ECF0>, search_kwargs={})

In [41]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool_obj = create_retriever_tool(
    retriever, 
    "langsmith_search", 
    "Search for information about LangSmith. For any question about LangSmith you must use this tool"
)


In [42]:
retriever_tool.name

'Retriever'

In [16]:
##ARxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv= ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name



'arxiv'

In [43]:
from langchain.tools import Tool

wiki_tool = Tool(
    name="Wikipedia",
    func=wiki.run,
    description="Useful for looking up general knowledge about people, places, or things."
)

arxiv_tool = Tool(
    name="Arxiv",
    func=arxiv.run,
    description="Useful for searching research papers on arXiv."
)

retriever_tool = Tool(
    name="Retriever",
    func=retriever_tool_obj.run,
    description="Useful for retrieving documents from a vectorstore."
)

tools = [wiki_tool, arxiv_tool, retriever_tool]


In [44]:
tools

[Tool(name='Wikipedia', description='Useful for looking up general knowledge about people, places, or things.', func=<bound method BaseTool.run of WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\genai\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))>),
 Tool(name='Arxiv', description='Useful for searching research papers on arXiv.', func=<bound method BaseTool.run of ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))>),
 Tool(name='Retriever', description='Useful for retrieving documents from a vectorstore.', func=<bound method BaseTool.run of Tool(name='langsmith

In [23]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma:2b")

C:\Users\hp\AppData\Local\Temp\ipykernel_85020\3720262705.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b")


In [45]:
from langchain import hub

prompt = hub.pull("hwchase17/structured-chat-agent")

print(type(prompt)) 
print(prompt.input_variables)


<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']


In [46]:
##agents: is to use a language model to choose a sequence of actions to take.
from langchain.agents import create_structured_chat_agent, AgentExecutor

agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


In [ ]:

def run_agent(input_text):
    result = agent_executor.invoke({"input": input_text, "chat_history": []})
    print(result["output"])



In [48]:
run_agent("Tell me about machine learning")



> Entering new AgentExecutor chain...
```json
{
  "action": "Final Answer",
  "action_input": "The history of Machine Learning is marked by the contributions of several pioneers, including Arthur Samuel, who developed the first self-learning checkers program in 1959. This work laid the foundation for modern artificial intelligence and continues to influence the field today."
}
```

> Finished chain.
The history of Machine Learning is marked by the contributions of several pioneers, including Arthur Samuel, who developed the first self-learning checkers program in 1959. This work laid the foundation for modern artificial intelligence and continues to influence the field today.
